# Pure repeats catalog creation

Hope Tanudisastro | Jan 9, 2023

The purpose of this notebook is to filter loci defined in the Illumina Repeats Catalog and keep only those with 100% pure repeat sequences.(https://github.com/Illumina/RepeatCatalogs). 

Repeat purity will be identified by TandemRepeatFinder (TRF). Loci that are not able to be analysed by TRF (due to short sequence length) will be filtered using a simple script/manually. 


### File importing and data cleaning

We first import the file containing the loci analysed by TRF. For details on how the TRF output file was generated, see here: https://docs.google.com/document/d/1C4Q6tFDYFWK3MJKeKjQavK6gug7NNa1TIpaGJy67ES4/edit?usp=sharing

In [1]:
import json, csv, re
f = open('trf_output.dat')
trf_output = f.readlines()

Example output: (line 8 is the first informative line)

In [2]:
for i in range(8,50):
    print(trf_output[i])

Sequence: chrX:148500631-148500691







Parameters: 2 7 7 80 10 2 500





1 60 3 20.0 3 92 0 102 0 63 33 3 1.11 GCC GCCGCTGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCTGCCGCC





Sequence: chrX:67545316-67545385







Parameters: 2 7 7 80 10 2 500





1 69 3 23.0 3 100 0 138 33 33 33 0 1.58 GCA GCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCA





Sequence: chr12:6936716-6936773







Parameters: 2 7 7 80 10 2 500





1 57 3 19.0 3 92 0 96 36 33 29 0 1.58 CAG CAGCAACAGCAACAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAG





Sequence: chr6:16327633-16327723







Parameters: 2 7 7 80 10 2 500





1 90 3 30.0 3 95 0 162 2 31 33 33 1.70 TGC TGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGATGCTGATGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGC





Sequence: chr22:45795354-45795424





We break up the TRF output file into sequence, parameter, and metrics arrays

In [3]:
sequence = [] #loci coordinates
parameter =[] #TRF parameter settings, will not be used for downstream analysis
metrics = [] #TRF metrics output for each locus, including repeat purity
i = 8 

while i<len(trf_output):
    if "Sequence:" in trf_output[i]:
        sequence.append(trf_output[i])
    elif "Parameters:" in trf_output[i]: 
        parameter.append(trf_output[i])
        i+=3
        metrics.append(trf_output[i])
    i+=1
        

Sanity checks: 

In [4]:
print(len(sequence))
print(len(parameter))
print(len(metrics))

174287
174287
174287


The Illumina catalog has 174, 393 loci. We removed 6 loci containing compound repeats prior to this analysis, so the 3 arrays above have the length we expect. 

Example output from each array:

In [5]:
for i in range(0,2): 
    print(sequence[i])
    print(parameter[i])
    print(metrics[i])

Sequence: chrX:148500631-148500691

Parameters: 2 7 7 80 10 2 500

1 60 3 20.0 3 92 0 102 0 63 33 3 1.11 GCC GCCGCTGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCTGCCGCC

Sequence: chrX:67545316-67545385

Parameters: 2 7 7 80 10 2 500

1 69 3 23.0 3 100 0 138 33 33 33 0 1.58 GCA GCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCA



The 3 arrays have the content we expect them to have. 

Some loci were not analysed by TandemRepeatFinder because their sequence was too short. These loci will have only a new line character as their entry in the metrics field. We will filter out these entries and deal with them later. 

In [6]:
blank_metrics_index=[]
i=0
while i<len(metrics):
    if metrics[i] =="\n":
        blank_metrics_index.append(i)
    i+=1

In [7]:
print(len(blank_metrics_index))

6409


Out of the 174287 loci analysed, 6409 loci have sequences that are too short to be analysed by TRF, and thus will need to be checked for repeat purity using an alternative method. 

### TRF loci filtering

We will first look at loci analysed by TRF. 

In [8]:
TRF_sequence = [] #loci coordinates
TRF_repeat_purity = [] #repeat purity associated with locus (value ranges from 0-100)

j=0 
while j< len(sequence):
    if j not in blank_metrics_index: 
        TRF_sequence.append((sequence[j])[10:]) #removes "Sequence:" from the coordinates string
        TRF_repeat_purity.append(metrics[j].split()[5]) #extracts the repeat purity value from the metrics output
    j+=1
        

Sanity check

In [9]:
print(len(TRF_sequence))
print(len(TRF_repeat_purity))

167878
167878


There are 167, 878 loci analysed by TRF.

Example output of arrays:

In [10]:
for i in range(0,2):
    print(TRF_sequence[i])
    print(TRF_repeat_purity[i])

chrX:148500631-148500691

92
chrX:67545316-67545385

100


Now we will filter the loci identified by TRF according to repeat purity. Only loci with 100% repeat purity will be kept. 

In [11]:
k= 0
pure_repeats_index =[] #stores indices of entries with pure repeats 
while k<len(TRF_repeat_purity):
    if TRF_repeat_purity[k]=="100": 
        pure_repeats_index.append(k)
    k+=1
        

In [12]:
l=0
coordinates_of_pure_repeats =[] #stores coordinates of sequences with pure repeats
while l<len(pure_repeats_index):
    coordinates_of_pure_repeats.append(TRF_sequence[pure_repeats_index[l]])
    l+=1
        


In [13]:
print(len(pure_repeats_index))

159812


In [14]:
print(len(coordinates_of_pure_repeats))

159812


There are 159,812 pure repeat loci, as identified by TRF. Their coordinates are stored in `coordinates_of_pure_repeats`.

### Purity filtering of loci not assessed by TRF

Now we will assess the loci that were not assessed by TRF because their sequence length was too short. 

In [15]:
blank_coordinates =[] #stores the coordinates of loci not assess with TRF (ie entries with blank metrics)
j=0 
while j< len(sequence):
    if j  in blank_metrics_index: 
        blank_coordinates.append((sequence[j])[10:].rstrip()) #remove "Sequence:" from the start of each string, and new line char from end of string
    j+=1

In [16]:
print(len(blank_coordinates))

6409


There are 6,409 loci that were not assessed by TRF, and thus need to be assessed for purity in this section.

To assess for purity of these sequences, we will consider whether the reference FASTA sequence is made up of perfect copies of the motif. We will consider loci that failed this filter on a case-by-case basis. 

Broadly we will: 
- Use the Illumina BED catalog to create a `motif_dictionary` that maps the coordinates of each locus to their respective motif. 
- Create a `fasta_dictionary` that maps the coordinates of each locus to their respective fasta sequence. 
- Assess each of the 6,409 unassessed loci to see if their fasta sequence (from `fasta_dictionary`) is made up of perfect copies of the motif (from `motif_dictionary`). 

#### Motif dictionary creation (key: genomic coordinates, value: motif)

In [17]:
b = open('bed_catalog_without_complex_repeats.bed')

In [18]:
bed_catalog = b.readlines()

In [19]:
motif_dictionary = {}
i = 0
while i < len(bed_catalog):
    bed_line = bed_catalog[i].split("\t")
    coordinates = bed_line[0]+":"+bed_line[1]+"-"+bed_line[2] #concatenates the chromosome, start_coordinate, and end_coordinate information
    motif = bed_line[4].rstrip() #remove new line character
    motif_dictionary[coordinates] = motif
    i+=1

In [20]:
len(motif_dictionary)

174287

Example output

In [21]:
print(motif_dictionary["chr9:27573528-27573546"])

GGCCCC


#### Fasta dictionary creation (key: genomic coordinates, value: FASTA sequence)

In [22]:
d = open('catalog_fasta_sequences.fasta.txt')

In [23]:
fasta = d.readlines()

In [24]:
fasta_dictionary = {}
m=0 
while m<len(fasta):
    if ">chr" in fasta[m]:
        coordinates = fasta[m][1:].rstrip() #removes the ">" from the coordinates entry
        m+=1
        fasta_seq = fasta[m].rstrip() #removes new line char
        fasta_dictionary[coordinates] = fasta_seq
    m+=1
        

Example usage

In [25]:
print(fasta_dictionary["chr12:6936716-6936773"])
print(motif_dictionary["chr12:6936716-6936773"])


CAGCAACAGCAACAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAG
CAG


#### Assessment of the 6,409 loci

As a broad filter with high specificity, we will first check if the reference sequence is made up of perfect copies of the motif at each locus.

In [26]:
impure_repeats_coordinates=[] #stores coordinates of impure repeats 
for i in blank_coordinates: #blank_coordinates contains coordinates of loci not assessed by TRF (ie those that have blank metrics)
    motif = motif_dictionary[i]
    fasta = fasta_dictionary[i]
    num_repeats = len(fasta)/len(motif)
    simulated_fasta = ""
    for x in range(int(num_repeats)): #creates a simulated sequence made up of repeating motifs
        simulated_fasta = simulated_fasta+motif 
    if simulated_fasta != fasta: 
        impure_repeats_coordinates.append(i)
    else: 
        coordinates_of_pure_repeats.append(i) #adds to existing array of coordinates of pure repeats as identified with TRF purity score
    
    

In [27]:
print(len(impure_repeats_coordinates))

42


In [28]:
print(len(coordinates_of_pure_repeats))

166179


This filter was able to identify that most of the loci were pure repeats, and there were only 42 remaining impure loci to look at manually. 

#### Manual filtering of remaining 42 loci

We will look at each entry manually to see if any contain permutations of pure repeats (eg instead of ATG x3, it is TGAx3)

In [29]:
for i in impure_repeats_coordinates:
    print(fasta_dictionary[i])
    print (motif_dictionary[i])

TCACTCATTCACTCAC
TCAC
TTATTTAATTATTTAT
TTAT
AATAAAAATAAT
AAT
TTTGTTTCTTTGTTTG
TTTG
CGCCGTCGCCGC
CGC
AATGAATCAATGAATG
AATG
TGTTGTTATTGT
TGT
AAGAAAGGAAGAAAGA
AAGA
TTTGTTTATTTGTTTG
TTTG
TTTATTTGTTTATTTA
TTTA
AAACAAAGAAACAAAC
AAAC
GGCGGGCAGGCGGGCG
GGCG
CAGCAGAAGCAG
CAG
AAACAAATAAACAAAC
AAAC
GAAGAAGGAGAA
GAA
TATTTATGTATTTATT
TATT
ATTAATTGATTAATTA
ATTA
TTGTTTGCTTGTTTGT
TTGT
GGAGGGAGCGAGGGAG
GGAG
AACAAAAACAAC
AAC
TTTATTTCTTTATTTA
TTTA
CTATCTACCTATCTAT
CTAT
TTTATTTCTTTATTTA
TTTA
GTTTGTTCGTTTGTTT
GTTT
AAACAAATAAACAAAC
AAAC
TAAATAAGTAAATAAA
TAAA
AAACAAATAAACAAAC
AAAC
CATTCATTAATTCATT
CATT
TTTCTTTATTTCTTTC
TTTC
CCACCTCCACCA
CCA
TCATTCACTCATTCAT
TCAT
CTTCCTTTCTTCCTTC
CTTC
GTTTGTTCGTTTGTTT
GTTT
TATCTATGTATCTATC
TATC
AAAGAAATAAAGAAAG
AAAG
AACAAACGAACAAACA
AACA
TTTATTTGTTTATTTA
TTTA
TTATTTACTTATTTAT
TTAT
AATGAATAAATGAATG
AATG
TTTGTTTATTTGTTTG
TTTG
AAACAAAGAAACAAAC
AAAC
TTTGTTTCTTTGTTTG
TTTG


All the 42 loci above contain indels and are therefore confirmed impure repeats. 

### Finishing up 

In summary we have parsed the Illumina 174k loci catalog for pure repeats. Repeat purity was defined preferentially by TRF's purity metric. If the TRF metric was unavaiable, purity was defined as whether the locus' FASTA sequence was made up of perfect copies of the motif. 

In [30]:
len(coordinates_of_pure_repeats)

166179

In total, there are 166, 179 loci made up of pure repeats. These loci will be used for benchmarking analysis. 